# mounting drive for data access

In [2]:
from google.colab import drive  
drive._mount('/content/drive') 

ValueError: ignored

# installing libraries


In [6]:
!pip install cdsapi
!pip install terality

  Created wheel for cdsapi: filename=cdsapi-0.5.1-py2.py3-none-any.whl size=11699 sha256=4b4b92feee102b1a3da01080ee9ddb206e878bde245bcdf677ea9f780aa533f4
  Stored in directory: /root/.cache/pip/wheels/1d/2e/3c/3746e0cd076320584d6f47b688da7b13a5d5d1a92606779ca4
Successfully built cdsapi
     |████████████████████████████████| 98 kB 3.3 MB/s 
     |████████████████████████████████| 131 kB 40.7 MB/s 
     |████████████████████████████████| 25.6 MB 112.8 MB/s 
     |████████████████████████████████| 97 kB 5.6 MB/s 
     |████████████████████████████████| 142 kB 53.2 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 138 kB 58.9 MB/s 
     |████████████████████████████████| 10.9 MB 43.5 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 8.4 MB 46.8 MB/s 
     |████████████████████████████████| 1.1 MB 70.4 MB/s 
     |████████████████████████████████| 144 kB 62.7 MB/s 
     |████████████████

In [7]:
import cdsapi
import netCDF4 as nc
from netCDF4 import num2date
from netCDF4 import Dataset
import numpy as np
import os
import terality as te
import pandas as pd
import xarray as xr

In [10]:
!terality account configure --email seafishleo@gmail.com

Your Terality API key: tla_YEq7jxFDoifhDvJzy3O1qzEX608pjU0AIe7z3Ys7ThgNFPeOQSId6PN4wAwJ12ys1q
Terality account succesfully configured on this system.




# Read Data from Drive 

In [11]:
my_nc_file = '/content/drive/MyDrive/data/ERA5_monthly_north.csv'
north_data = te.read_csv(my_nc_file)
north_data

UnconfiguredTeralityClientError: ignored

In [ ]:
# USING XARRAY
#my_nc_file = '/content/drive/MyDrive/data/ERA5_monthly_north.nc'
#ds = xr.open_dataset(my_nc_file)
#north_data = ds.to_dataframe()
#north_data.head(100)

In [ ]:
north_data.dtypes

INFO:terality:The result of te.dataframe.dtypes was retrieved from cache [docs: https://docs.terality.com/getting-terality/user-guide/caching].


expver         int64
latitude     float64
longitude    float64
time          object
t2m          float64
siconc       float64
asn          float64
smlt         float64
sp           float64
tco3         float64
dtype: object

In [ ]:
len(north_data)

141177600

In [ ]:
north_data.count()

expver       141177600
latitude     141177600
longitude    141177600
time         141177600
t2m           70588800
siconc        15390216
asn           70588800
smlt          70588800
sp            70588800
tco3          70588800
dtype: int64

# Rename  & Calculation

In [ ]:
north_data__renamed = north_data.rename(columns={'t2m':'temperature_2m','siconic':'sea_ice_area_fraction','asn':'Snow_albedo','smlt':'snow_melt','sp':'surface_pressure','tco3':
'Total_column_ozone'})

In [ ]:
north_data__renamed = north_data__renamed.drop(['expver','latitude','longitude'],axis=1)
north_data__renamed

In [ ]:
north_avg = north_data__renamed.groupby('time').mean()

In [ ]:
north_avg

,temperature_2m,siconc,Snow_albedo,snow_melt,surface_pressure,Total_column_ozone
time,,,,,,
1979-01-01,255.527956,0.393509,0.856535,3.674974e-06,80884.359104,0.006313
1979-02-01,249.151987,0.268763,0.856540,3.973923e-08,80559.107569,0.006019
1979-03-01,242.581619,0.347582,0.856535,9.692198e-09,80029.685869,0.006080
1979-04-01,239.475744,0.613826,0.856540,5.515169e-09,80135.939285,0.005907
1979-05-01,235.630476,0.814768,0.856545,4.656613e-10,79953.811282,0.005999
...,...,...,...,...,...,...
2021-08-01,233.059227,0.927768,0.856397,4.656613e-10,79942.072208,0.005444
2021-09-01,235.898869,0.920421,0.856432,4.656613e-10,79449.651808,0.004006
2021-10-01,240.550710,0.895792,0.856416,4.656613e-10,79909.074749,0.003351


In [ ]:
north_avg.dtypes

temperature_2m        float64
siconc                float64
Snow_albedo           float64
snow_melt             float64
surface_pressure      float64
Total_column_ozone    float64
dtype: object

# Load Seaice Extent Dataframe

In [ ]:
n_seaice_df = pd.read_csv('/content/drive/MyDrive/data/N_seaice_extent_daily_v3.0.csv')
n_seaice_df

,Year,Month,Day,Extent,Missing,Source Data
0,YYYY,MM,DD,10^6 sq km,10^6 sq km,Source data product web sites: http://nsidc.o...
1,1978,10,26,17.624,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.26/nt_19...
2,1978,10,28,17.803,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.28/nt_19...
3,1978,10,30,17.670,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.30/nt_19...
4,1978,11,01,17.527,0.000,['/ecs/DP1/PM/NSIDC-0051.001/1978.11.01/nt_19...
...,...,...,...,...,...,...
14124,2022,01,04,4.988,0.000,['/ecs/DP1/PM/NSIDC-0081.001/2022.01.03/nt_202...
14125,2022,01,05,4.819,0.000,['/ecs/DP1/PM/NSIDC-0081.001/2022.01.04/nt_202...
14126,2022,01,06,4.686,0.000,['/ecs/DP1/PM/NSIDC-0081.001/2022.01.05/nt_202...
14127,2022,01,07,4.634,0.000,['/ecs/DP1/PM/NSIDC-0081.001/2022.01.06/nt_202...


In [ ]:
n_seaice_df.drop([' Source Data','    Missing',' Day'], axis = 1, inplace = True)
n_seaice_df = n_seaice_df[n_seaice_df.Year != '1978']
n_seaice_df = n_seaice_df[n_seaice_df.Year != '2022']
n_seaice_df = n_seaice_df.iloc[1: , :]
n_seaice_df.reset_index(drop=True, inplace=True)
n_seaice_df

,Year,Month,Extent
0,1979,01,6.945
1,1979,01,6.838
2,1979,01,6.638
3,1979,01,6.270
4,1979,01,6.138
...,...,...,...
14081,2021,12,6.461
14082,2021,12,6.295
14083,2021,12,6.185
14084,2021,12,5.846


In [ ]:
n_seaice_df.dtypes

Year           object
 Month         object
     Extent    object
dtype: object

In [ ]:
n_seaice_df["time"] = n_seaice_df["Year"].astype(str) + '-' + n_seaice_df[" Month"].str.strip()+'-01'
n_seaice_df['     Extent'] = n_seaice_df['     Extent'].astype(float)
n_seaice_df.drop(['Year',' Month'], axis = 1, inplace = True)
n_seaice_df

,Extent,time
0,6.945,1979-01-01
1,6.838,1979-01-01
2,6.638,1979-01-01
3,6.270,1979-01-01
4,6.138,1979-01-01
...,...,...
14081,6.461,2021-12-01
14082,6.295,2021-12-01
14083,6.185,2021-12-01
14084,5.846,2021-12-01


In [ ]:
n_seaice_avg = n_seaice_df.groupby('time').mean()
n_seaice_avg

,Extent
time,
1979-01-01,5.402200
1979-02-01,3.141071
1979-03-01,3.999312
1979-04-01,7.485533
1979-05-01,10.826667
...,...
2021-08-01,18.130871
2021-09-01,18.451467
2021-10-01,17.626000


# Join Two Dataframes

In [ ]:
north_joined_df = north_avg.join(n_seaice_avg)
north_joined_df

,temperature_2m,siconc,Snow_albedo,snow_melt,surface_pressure,Total_column_ozone,Extent
time,,,,,,,
1979-01-01,255.527956,0.393509,0.856535,3.674974e-06,80884.359104,0.006313,5.402200
1979-02-01,249.151987,0.268763,0.856540,3.973923e-08,80559.107569,0.006019,3.141071
1979-03-01,242.581619,0.347582,0.856535,9.692198e-09,80029.685869,0.006080,3.999312
1979-04-01,239.475744,0.613826,0.856540,5.515169e-09,80135.939285,0.005907,7.485533
1979-05-01,235.630476,0.814768,0.856545,4.656613e-10,79953.811282,0.005999,10.826667
...,...,...,...,...,...,...,...
2021-08-01,233.059227,0.927768,0.856397,4.656613e-10,79942.072208,0.005444,18.130871
2021-09-01,235.898869,0.920421,0.856432,4.656613e-10,79449.651808,0.004006,18.451467
2021-10-01,240.550710,0.895792,0.856416,4.656613e-10,79909.074749,0.003351,17.626000


In [26]:
north_joined_df = pd.read_csv('/content/drive/MyDrive/data/North_Data_Finalized.csv')
CO2_df = pd.read_csv('/content/drive/MyDrive/data/CO2.csv')
CO2_df["time"] = CO2_df["time"].astype(str)
methane_df = pd.read_csv('/content/drive/MyDrive/data/methane.csv')
methane_df["time"] = methane_df["time"].astype(str)
north_joined_df_new = pd.merge(north_joined_df, CO2_df, on="time")
north_joined_df_new = pd.merge(north_joined_df_new, methane_df, on="time")
north_joined_df_new.to_csv('/content/drive/MyDrive/data/North_Data_Finalized_1.csv')
north_joined_df_new

,time,temperature_2m,siconc,Snow_albedo,snow_melt,surface_pressure,Total_column_ozone,Extent,xco2,xch4
0,2003-02-01,246.741962,0.884758,0.838650,5.951018e-06,99292.290584,0.008306,15.193143,376.38867,1732.6155
1,2003-03-01,249.224691,0.892143,0.837207,4.105676e-06,99114.335720,0.008907,15.484903,374.52463,1736.9479
2,2003-04-01,256.826361,0.884645,0.840880,4.731454e-05,99689.368457,0.008886,14.512067,375.49440,1744.8373
3,2003-06-01,274.019265,0.780127,0.853892,5.575093e-04,99331.137871,0.007478,11.604033,374.63092,1744.2860
4,2003-08-01,274.893003,0.575636,0.869577,7.181390e-05,99127.633735,0.006325,6.935871,369.44266,1750.8135
...,...,...,...,...,...,...,...,...,...,...
194,2020-02-01,249.717996,0.864570,0.843789,5.274536e-07,98367.752806,0.007178,14.642138,410.65050,1832.6729
195,2020-03-01,250.632265,0.874701,0.845780,2.599765e-06,98420.999313,0.006682,14.730000,412.03122,1845.1708
196,2020-04-01,260.010986,0.872704,0.845278,6.161167e-05,99059.116570,0.007072,13.621500,413.05817,1850.6763
197,2020-05-01,268.472761,0.826479,0.844218,4.557183e-04,99821.752457,0.007833,12.343129,414.23456,1853.0302


In [27]:
south_joined_df = pd.read_csv('/content/drive/MyDrive/data/South_Data_Finalized.csv')
CO2_df = pd.read_csv('/content/drive/MyDrive/data/CO2.csv')
CO2_df["time"] = CO2_df["time"].astype(str)
methane_df = pd.read_csv('/content/drive/MyDrive/data/methane.csv')
methane_df["time"] = methane_df["time"].astype(str)
south_joined_df_new = pd.merge(south_joined_df, CO2_df, on="time")
south_joined_df_new = pd.merge(south_joined_df_new, methane_df, on="time")
south_joined_df_new.to_csv('/content/drive/MyDrive/data/South_Data_Finalized_1.csv')
south_joined_df_new

,time,temperature_2m,siconc,Snow_albedo,snow_melt,surface_pressure,Total_column_ozone,Extent,xco2,xch4
0,2003-02-01,250.531347,0.354018,0.856101,8.528879e-07,81072.590525,0.005571,3.788571,376.38867,1732.6155
1,2003-03-01,242.459735,0.451664,0.856246,3.343988e-07,80563.469941,0.005698,4.444355,374.52463,1736.9479
2,2003-04-01,237.807589,0.638975,0.856267,1.298517e-08,80001.156018,0.005574,7.208267,375.49440,1744.8373
3,2003-06-01,234.455553,0.871961,0.856296,4.656613e-10,80958.719176,0.005815,13.985367,374.63092,1744.2860
4,2003-08-01,234.623882,0.921810,0.856332,4.656613e-10,79701.416766,0.005420,17.582484,369.44266,1750.8135
...,...,...,...,...,...,...,...,...,...,...
194,2020-02-01,250.318563,0.204383,0.856185,7.597154e-06,80588.276313,0.006255,2.920069,410.65050,1832.6729
195,2020-03-01,242.411176,0.310277,0.856330,8.730145e-07,79871.878277,0.006216,4.002710,412.03122,1845.1708
196,2020-04-01,240.169114,0.555356,0.856409,5.129950e-09,80307.378069,0.005994,6.662367,413.05817,1850.6763
197,2020-05-01,238.458923,0.796888,0.856404,4.656613e-10,80111.997222,0.005931,9.871097,414.23456,1853.0302


In [ ]:
north_joined_df.to_csv('/content/drive/MyDrive/data/North_Data_Finalized1.csv')